In [21]:
import os
import sys
import json
import numpy as np
import pandas as pd

sys.path.append("../helpers/")
from datasets import *

data_path = "data"
datasets_path = "datasets"

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Datasets for table classification
In this notebook, NLL-tag and NLL-auto are constructed.

In [22]:
NLL_tag = pd.read_parquet(os.path.join(data_path, "NLL_tag.parquet"))
NLL_auto = pd.read_parquet(os.path.join(data_path, "NLL_auto.parquet"))

## Pre-processing

In [23]:
NLL_tag = NLL_tag[NLL_tag['tag'] != 'not a table']
NLL_tag.loc[NLL_tag['tag'].isin(['currency rates', 'stock']), 'tag'] = 'exchange'
NLL_tag.loc[NLL_tag['tag'] == 'sports results', 'tag'] = 'sport results'
NLL_tag.loc[NLL_tag['tag'] == 'table', 'tag'] = 'miscellaneous'

print(NLL_tag.shape)
NLL_tag.head()

(3783, 22)


,tag,date_tag,auto_tag,tb_coords,text_coords,pid,text,language,iiifURL,iiifFragmentURL,...,day,journal,edition,page,meta_issue_id,x,y,width,height,area
luxwort-1891-06-22-a-i0059,transport schedule,2021-11-04T18:49:59.000Z,False,"{'height': [70, 227, 987], 'width': [284, 543,...","{'height': [25, 14, 24, 27, 15, 22, 15, 23, 21...",luxwort-1891-06-22-a-p0004,abf. vom Abfahrt nfangs- nom End- -Uli nach A....,de,https://iiif.eluxemburgensia.lu/image/iiif/2/a...,[https://iiif.eluxemburgensia.lu/image/iiif/2/...,...,22,luxwort,a,4,luxwort-1891-06-22-a,3109,4926,544,1298,706112
luxwort-1882-07-14-a-i0026,transport schedule,2021-11-04T18:06:27.000Z,False,"{'height': [1185], 'width': [1111], 'x': [20],...","{'height': [30, 30, 14, 29, 14, 27, 15, 36, 12...",luxwort-1882-07-14-a-p0004,"»bs. VlUffel. « Namllr , »llon , Nettingen , C...",de,https://iiif.eluxemburgensia.lu/image/iiif/2/a...,[https://iiif.eluxemburgensia.lu/image/iiif/2/...,...,14,luxwort,a,4,luxwort-1882-07-14-a,20,3132,1111,1185,1316535
luxwort-1929-05-29-a-i0038,exchange,2021-10-12T18:11:31.000Z,False,"{'height': [225], 'width': [1002], 'x': [1543]...","{'height': [28, 22, 29, 22, 21, 21, 25, 22, 20...",luxwort-1929-05-29-a-p0007,Îelgo-Polonaise Soc. (Sobelpol) îruxell. Soc. ...,fr,https://iiif.eluxemburgensia.lu/image/iiif/2/a...,[https://iiif.eluxemburgensia.lu/image/iiif/2/...,...,29,luxwort,a,7,luxwort-1929-05-29-a,1543,3532,1002,225,225450
luxwort-1941-05-29-a-i0063,exchange,2021-10-13T16:29:30.000Z,False,"{'height': [587], 'width': [837], 'x': [2759],...","{'height': [21, 21, 18, 20, 21, 17, 47, 20, 35...",luxwort-1941-05-29-a-p0004,78. M tel Flf Äiitrtlitj t-M I tetd | »nef 1 i...,de,https://iiif.eluxemburgensia.lu/image/iiif/2/a...,[https://iiif.eluxemburgensia.lu/image/iiif/2/...,...,29,luxwort,a,4,luxwort-1941-05-29-a,2759,2937,837,587,491319
luxwort-1895-06-12-a-i0005,exchange,2021-10-12T18:35:50.000Z,False,"{'height': [2988], 'width': [894], 'x': [3135]...","{'height': [35, 27, 28, 35, 35, 28, 36, 35, 30...",luxwort-1895-06-12-a-p0003,Banque de France 9tei$«6anf Banque nationale B...,de,https://iiif.eluxemburgensia.lu/image/iiif/2/a...,[https://iiif.eluxemburgensia.lu/image/iiif/2/...,...,12,luxwort,a,3,luxwort-1895-06-12-a,3135,2815,894,2988,2671272


In [24]:
NLL_auto = NLL_auto[NLL_auto['tag'] != 'not a table']
NLL_auto.loc[NLL_auto['tag'].isin(['currency rates', 'stock']), 'tag'] = 'exchange'
NLL_auto.loc[NLL_auto['tag'] == 'sports results', 'tag'] = 'sport results'
NLL_auto.loc[NLL_auto['tag'] == 'table', 'tag'] = 'miscellaneous'

print(NLL_auto.shape)
NLL_auto.head()

(8531, 25)


,tag,date_tag,auto_tag,same_journal,timespan,og_id,tb_coords,text_coords,pid,text,...,day,journal,edition,page,meta_issue_id,x,y,width,height,area
luxwort-1890-11-12-a-i0017,food prices,2021-11-17T13:35:00.256Z,True,True,1825.0,luxwort-1891-01-10-a-i0011,"{'height': [561], 'width': [1024], 'x': [3291]...","{'height': [34, 28, 36, 29, 34, 27, 34, 37, 14...",luxwort-1890-11-12-a-p0003,gt. «t. 3fr. ©t. Leizen p« Heltol. »ischelfruc...,...,12,luxwort,a,3,luxwort-1890-11-12-a,3291,4238,1024,561,574464
luxwort-1891-06-22-a-i0059,transport schedule,2021-11-04T18:49:59.000Z,False,None,NaN,None,"{'height': [70, 227, 987], 'width': [284, 543,...","{'height': [25, 14, 24, 27, 15, 22, 15, 23, 21...",luxwort-1891-06-22-a-p0004,abf. vom Abfahrt nfangs- nom End- -Uli nach A....,...,22,luxwort,a,4,luxwort-1891-06-22-a,3109,4926,544,1298,706112
courriergdl-1864-10-07-a-i0017,transport schedule,2021-11-17T13:22:12.748Z,True,True,1825.0,courriergdl-1865-03-23-a-i0016,"{'height': [416], 'width': [3473], 'x': [83], ...","{'height': [36, 29, 25, 34, 7, 27, 7, 27, 28, ...",courriergdl-1864-10-07-a-p0001,Dép.deLuxemb. Arr. à Bettembg. — àTlùonville —...,...,7,courriergdl,a,1,courriergdl-1864-10-07-a,83,825,3473,416,1444768
waechtersauer-1855-08-25-a-i0008,food prices,2021-11-17T13:18:11.617Z,True,True,1825.0,waechtersauer-1855-07-25-a-i0012,"{'height': [405], 'width': [888], 'x': [2048],...","{'height': [38, 33, 34, 27, 34, 25, 29, 34, 25...",waechtersauer-1855-08-25-a-p0004,"Diekirch. Weizen. perHektl, 32.00 Mengtorn id....",...,25,waechtersauer,a,4,waechtersauer-1855-08-25-a,2048,3643,888,405,359640
courriergdl-1861-08-21-a-i0008,transport schedule,2021-11-17T13:18:47.956Z,True,True,1825.0,courriergdl-1861-08-20-a-i0016,"{'height': [218], 'width': [3416], 'x': [158],...","{'height': [31, 23, 22, 35, 28, 30, 16, 24, 25...",courriergdl-1861-08-21-a-p0001,Ligne de France. Départs de Luxembourg : 5 h. ...,...,21,courriergdl,a,1,courriergdl-1861-08-21-a,158,825,3416,218,744688


## NLL-tag: manually classified tables

In [5]:
print_tag_distributions(NLL_tag)

cinema                  4 items (0.1%) on    2 pages on    2 issues
election               49 items (1.3%) on   11 pages on   11 issues
exchange              444 items (11.7%) on  312 pages on  312 issues
food prices           177 items (4.7%) on  143 pages on  143 issues
lotto                   8 items (0.2%) on    8 pages on    8 issues
miscellaneous         365 items (9.6%) on  217 pages on  208 issues
radio                   3 items (0.1%) on    3 pages on    3 issues
sport results         403 items (10.7%) on   41 pages on   41 issues
transport schedule   2301 items (60.8%) on  408 pages on  407 issues
weather                29 items (0.8%) on   28 pages on   28 issues


In [6]:
NLL_tag.loc[NLL_tag['tag'].isin({'cinema', 'election', 'lotto', 'radio'}), 'tag'] = 'miscellaneous'
print_tag_distributions(NLL_tag)

exchange              444 items (11.7%) on  312 pages on  312 issues
food prices           177 items (4.7%) on  143 pages on  143 issues
miscellaneous         429 items (11.3%) on  237 pages on  228 issues
sport results         403 items (10.7%) on   41 pages on   41 issues
transport schedule   2301 items (60.8%) on  408 pages on  407 issues
weather                29 items (0.8%) on   28 pages on   28 issues


In [7]:
NLL_tag = NLL_tag[["text", "pid", "tag", "text_coords", "x", "y", "width", "height", "meta_issue_id"]]

We use the same splits that for the datasets on table detection.

In [8]:
filename = "via_annotations_NLL_tag"

with (open(os.path.join(datasets_path, filename + "_train.json"), "r")) as f:
    via_annotations_NLL_tag_train = json.load(f)

with (open(os.path.join(datasets_path, filename + "_val.json"), "r")) as f:
    via_annotations_NLL_tag_val = json.load(f)

with (open(os.path.join(datasets_path, filename + "_test.json"), "r")) as f:
    via_annotations_NLL_tag_test = json.load(f)

In [9]:
NLL_tag_train = NLL_tag[NLL_tag['pid'].isin(via_annotations_NLL_tag_train.keys())]
NLL_tag_val = NLL_tag[NLL_tag['pid'].isin(via_annotations_NLL_tag_val.keys())]
NLL_tag_test = NLL_tag[NLL_tag['pid'].isin(via_annotations_NLL_tag_test.keys())]

print_splits(NLL_tag_train, NLL_tag_val, NLL_tag_test)

Dataset: 3783 items on 984 pages.
-- Train set: 2354 items on 605 pages.
-- Validation set: 549 items on 173 pages.
-- Test set: 880 items on 206 pages.


In [10]:
print("__________________Train set__________________")
print_tag_distributions(NLL_tag_train)
print()
print("_______________Validation set________________")
print_tag_distributions(NLL_tag_val)
print()
print("__________________Test set___________________")
print_tag_distributions(NLL_tag_test)

__________________Train set__________________
exchange              265 items (11.3%) on  188 pages on  188 issues
food prices           108 items (4.6%) on   86 pages on   86 issues
miscellaneous         276 items (11.7%) on  145 pages on  142 issues
sport results         238 items (10.1%) on   25 pages on   25 issues
transport schedule   1452 items (61.7%) on  245 pages on  245 issues
weather                15 items (0.6%) on   14 pages on   14 issues

_______________Validation set________________
exchange               79 items (14.4%) on   55 pages on   55 issues
food prices            30 items (5.5%) on   28 pages on   28 issues
miscellaneous          82 items (14.9%) on   48 pages on   48 issues
sport results          37 items (6.7%) on    7 pages on    7 issues
transport schedule    318 items (57.9%) on   68 pages on   68 issues
weather                 3 items (0.5%) on    3 pages on    3 issues

__________________Test set___________________
exchange              100 items (11.4

### Export

In [11]:
filename = "NLL_tag"
NLL_tag_train.to_parquet(os.path.join(datasets_path, filename + "_train.parquet"))
NLL_tag_val.to_parquet(os.path.join(datasets_path, filename + "_val.parquet"))
NLL_tag_test.to_parquet(os.path.join(datasets_path, filename + "_test.parquet"))

In [12]:
filename = "NLL_tag"
NLL_tag_train = pd.read_parquet(os.path.join(datasets_path, filename + "_train.parquet"))
NLL_tag_val = pd.read_parquet(os.path.join(datasets_path, filename + "_val.parquet"))
NLL_tag_test = pd.read_parquet(os.path.join(datasets_path, filename + "_test.parquet"))

## NLL-auto: automatically classified tables

In [25]:
print_tag_distributions(NLL_auto)

cinema                  4 items (0.0%) on    2 pages on    2 issues
election               49 items (0.6%) on   11 pages on   11 issues
exchange             1123 items (13.2%) on  874 pages on  874 issues
food prices           616 items (7.2%) on  569 pages on  569 issues
lotto                  13 items (0.2%) on   13 pages on   13 issues
miscellaneous         392 items (4.6%) on  242 pages on  233 issues
radio                   3 items (0.0%) on    3 pages on    3 issues
sport results         428 items (5.0%) on   58 pages on   58 issues
transport schedule   5868 items (68.8%) on 3062 pages on 3060 issues
weather                35 items (0.4%) on   34 pages on   34 issues


In [26]:
NLL_auto.loc[NLL_auto['tag'].isin({'cinema', 'election', 'lotto', 'radio'}), 'tag'] = 'miscellaneous'
print_tag_distributions(NLL_auto)

exchange             1123 items (13.2%) on  874 pages on  874 issues
food prices           616 items (7.2%) on  569 pages on  569 issues
miscellaneous         461 items (5.4%) on  267 pages on  258 issues
sport results         428 items (5.0%) on   58 pages on   58 issues
transport schedule   5868 items (68.8%) on 3062 pages on 3060 issues
weather                35 items (0.4%) on   34 pages on   34 issues


In [27]:
NLL_auto = NLL_auto[["text", "pid", "auto_tag", "tag", "text_coords", "x", "y", "width", "height", "meta_issue_id"]]

In [ ]:
NLL_auto_train, NLL_auto_val, NLL_auto_test = stratified_split_by_id(NLL_auto, [0.6, 0.2, 0.2], manual_test_tag=True)

In [40]:
print_splits(NLL_auto_train, NLL_auto_val, NLL_auto_test)

Dataset: 8531 items on 4616 pages.
-- Train set: 5120 items on 3559 pages.
-- Validation set: 1702 items on 1438 pages.
-- Test set: 1709 items on 681 pages.


In [41]:
print("__________________Train set__________________")
print_tag_distributions(NLL_auto_train)
print()
print("_______________Validation set________________")
print_tag_distributions(NLL_auto_val)
print()
print("__________________Test set___________________")
print_tag_distributions(NLL_auto_test)

__________________Train set__________________
exchange              674 items (13.2%) on  569 pages on  569 issues
food prices           370 items (7.2%) on  360 pages on  360 issues
miscellaneous         277 items (5.4%) on  186 pages on  184 issues
sport results         257 items (5.0%) on   48 pages on   48 issues
transport schedule   3521 items (68.8%) on 2459 pages on 2458 issues
weather                21 items (0.4%) on   21 pages on   21 issues

_______________Validation set________________
exchange              224 items (13.2%) on  214 pages on  214 issues
food prices           122 items (7.2%) on  120 pages on  120 issues
miscellaneous          91 items (5.3%) on   75 pages on   74 issues
sport results          85 items (5.0%) on   34 pages on   34 issues
transport schedule   1173 items (68.9%) on 1002 pages on 1001 issues
weather                 7 items (0.4%) on    7 pages on    7 issues

__________________Test set___________________
exchange              225 items (13.2%) 

### Export

In [ ]:
filename = "NLL_auto"
NLL_auto_train.to_parquet(os.path.join(datasets_path, filename + "_train.parquet"))
NLL_auto_val.to_parquet(os.path.join(datasets_path, filename + "_val.parquet"))
NLL_auto_test.to_parquet(os.path.join(datasets_path, filename + "_test.parquet"))

In [39]:
filename = "NLL_auto"
NLL_auto_train = pd.read_parquet(os.path.join(datasets_path, filename + "_train.parquet"))
NLL_auto_val = pd.read_parquet(os.path.join(datasets_path, filename + "_val.parquet"))
NLL_auto_test = pd.read_parquet(os.path.join(datasets_path, filename + "_test.parquet"))